# Waterbodies

* **Products used:** [DE Africa Waterbodies](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html)

## Background
It is important to have up-to-date information about the extent and location of surface water across Africa. 
It provides us with a common understanding of this valuable and increasingly scarce resource.

### What this product offers
The tool uses a water classification for every available Landsat satellite image and maps the locations of water bodies across Africa. It provides a time series of wet, dry and invalid (not wet or dry) surface area, for water bodies that are present more than 10% of the time and are larger than 4,500 m$^2$ (5 Landsat pixels).

The tool indicates changes in the wet surface area of water bodies. This can be used to identify when water bodies are increasing or decreasing in wet surface area.

**Disclaimer: DE Africa Water Bodies graphs show the wet surface area of waterbodies as estimated from satellites. They do not show depth, volume, purpose of the waterbody, nor the source of the water.**

The product contains polygons of maximum extent and surface area time series for each waterbody. These water bodies have been derived from [DE Africa Water Observations from Space](https://explorer.digitalearth.africa/products/wofs_ls), a classifier which detects open water in Landsat pixels.

### Applications 
* Identify and analyse persistent water bodies 
* Water resource management 
* Gain insights into the severity and spatial distribution of drought

## Description
This notebook will demonstrate how to load data from DE Africa Water Bodies using the scripts in this repository.
Topics covered include:

1. Getting a list of available polygons in a given area
2. Select and plot a single polygon
3. Plotting the surface area of the polygon over time

***

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
# !python -m pip install --extra-index-url="https://packages.dea.ga.gov.au" -e ../Tools/

In [ ]:
import matplotlib.pyplot as plt
import sys 
# sys.path.insert(1, "../Tools/")
from deafrica_tools.plotting import display_map
from deafrica_tools.waterbodies import (
    get_geohashes,
    get_waterbodies,
    get_waterbody,
    get_time_series,
)

In [ ]:
import geopandas as gpd
from owslib.wfs import WebFeatureService
from owslib.fes import PropertyIsEqualTo
from owslib.etree import etree
import pandas as pd

## Analysis parameters

Choose an area to analyse:

In [ ]:
lat, lon = 16.15335, -12.60475 #15.53457, -12.69639 #15.30869, -14.44771 16.15335°N, 12.60475°W
buffer = 0.2
xlim = (lon-buffer, lon+buffer)
ylim =  (lat+buffer, lat-buffer)

# Preview area on a map
display_map(xlim, ylim)

## Getting data

The `deafrica_waterbodies` module allows you to query water bodies by location or geohash.

### List waterbody polygons in an area
We can get a list of waterbody polygons inside a bounding box of coordinates using `get_waterbodies`.

In [ ]:
# Create a bounding box from study area coordinates
bbox = (xlim[0], ylim[1], xlim[1], ylim[0])

# Select all water bodies located within the bounding box
polygons = get_waterbodies(bbox, crs="EPSG:4326")

In [ ]:
# Return GeoDataFrame with selected water bodies
polygons

The returned geodataframe includes all the waterbodies which are located within the bounding box. This dataset contains metadata for each waterbody in the dataset, including the ID, UID, WB_UID, area, perimeter and time series. 

#### Geohashes
Every waterbody in DE Africa Water Bodies has a unique identifier (UID) called a [_geohash_](https://en.wikipedia.org/wiki/Geohash). The geohash of a waterbody is derived from their position, and this process can be reversed to obtain the location from the geohash. A waterbody's geohash is contained under the UID attribute and can be obtained through [DE Africa Maps](https://maps.digitalearth.africa/) by clicking on a waterbody.

**Note:** You might notice that these polygons have a range of similar geohash prefixes ('edyg' or 'edz'). If two geohashes have a similar prefix, this means that they are close together.

In [ ]:
# Plot the waterbody polygons located within the bounding box
polygons.plot()

We can use the `.explore()` function to interactively explore the subset of water bodies located within the bounding box. This allows us to hover over a given waterbody and extract any information, such as a waterbody's UID which we can use in as a input in future steps.

In [ ]:
# Explore the waterbody polygons located within the bounding box
polygons.explore()

### Getting data for a specific waterbody

We can use a geohash to get the polygon for an individual waterbody, using the `get_waterbody` function.

For example, we can load data for using its geohash `edz5cm96j`.

In [ ]:
selected_waterbody = get_waterbody("edz5cm96j")#"edz5cm93z"
selected_waterbody

In [ ]:
selected_waterbody.plot()

For any given geohash or a polygon, we can also use the `get_time_series` function to get surface area over time.

In [ ]:
selected_waterbody.timeseries.item()

In [ ]:
get_time_series(waterbody=selected_waterbody.iloc[0])["px_wet"].plot() #(waterbody=polygons.iloc[42])
plt.ylabel("Number of wet pixels");